In [1]:
import os
import gc
import math
import json
from datetime import datetime
import shutil
from pathlib import Path
import optuna
import numpy as np
import pandas as pd
import pytorch_lightning as pl
from tqdm import tqdm
import torch
from sklearn.model_selection import StratifiedGroupKFold
from torch.utils.data import DataLoader
from typing import Dict, List, Tuple, NamedTuple
from transformers import AutoTokenizer, AutoConfig, BertTokenizerFast
from transformers import BertConfig, RobertaConfig, EncoderDecoderConfig, EncoderDecoderModel
import scml

In [2]:
class ModelConf(NamedTuple):
    directory: Path
    model_max_length: int
    stride: int
    batch_size: int
    gradient_checkpointing: bool


class Conf(NamedTuple):
    model_name: str = "roberta-base"
    epochs: int = 3
    lr: Tuple[float, float] = (2e-6, 2e-6)
    multi_sample_dropout_size: Tuple[int, int] = (8, 8)
    multi_sample_dropout_increment: Tuple[float, float] = (0, 0)
    swa_start_epoch: Tuple[int, int] = (-1, -1)
    swa_anneal_epochs: Tuple[int, int] = (3, 3)
    pretrained_dir: Path = Path("pretrained")
    zoo: Dict[str, ModelConf] = {
        "deberta-v3-base": ModelConf( # sentencepiece tokenizer
            directory=pretrained_dir / "microsoft" / "deberta-v3-base",
            model_max_length=512,
            stride=128,
            batch_size=2,
            gradient_checkpointing=False
        ),
        "roberta-base": ModelConf( # bbpe tokenizer
            directory=pretrained_dir / "roberta-base",
            model_max_length=512,
            stride=128,
            batch_size=4,
            gradient_checkpointing=False
        )
    }
    sample_frac: float = 1e-5
    gpus: List[int] = [0]
    patience: int = 1
    n_trials: int = 1
    n_folds: int = 3
    seed: int = 31
        
        
conf = Conf()
mc = conf.zoo[conf.model_name]
print(conf)

Conf(model_name='roberta-base', epochs=3, lr=(2e-06, 2e-06), multi_sample_dropout_size=(8, 8), multi_sample_dropout_increment=(0, 0), swa_start_epoch=(-1, -1), swa_anneal_epochs=(3, 3), pretrained_dir=WindowsPath('pretrained'), zoo={'deberta-v3-base': ModelConf(directory=WindowsPath('pretrained/microsoft/deberta-v3-base'), model_max_length=512, stride=128, batch_size=2, gradient_checkpointing=False), 'roberta-base': ModelConf(directory=WindowsPath('pretrained/roberta-base'), model_max_length=512, stride=128, batch_size=4, gradient_checkpointing=False)}, sample_frac=1e-05, gpus=[0], patience=1, n_trials=1, n_folds=3, seed=31)


In [3]:
tim = scml.Timer()
tim.start()
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
os.environ["TOKENIZERS_PARALLELISM"] = "false"
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything(conf.seed)
pl.seed_everything(conf.seed)

Global seed set to 31


31

In [4]:
device: torch.device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
    for i in range(torch.cuda.device_count()):
        print(f"device={i}, {torch.cuda.get_device_name(i)}")
        print('Mem Allocated:', round(torch.cuda.memory_allocated(i)/1024**3,1), 'GB')
        print('Mem Cached:   ', round(torch.cuda.memory_reserved(i)/1024**3,1), 'GB')
else:
    print("cpu")

device=0, NVIDIA GeForce GTX 1060 6GB
Mem Allocated: 0.0 GB
Mem Cached:    0.0 GB


In [5]:
%%time
df = pd.read_parquet("input/sequences.parquet")
if conf.sample_frac<1:
    df = df.sample(frac=conf.sample_frac)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 129 entries, 2326609 to 6185607
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   session  129 non-null    int32 
 1   seq      129 non-null    object
 2   length   129 non-null    int16 
dtypes: int16(1), int32(1), object(1)
memory usage: 2.8+ KB
Wall time: 14.4 s


In [6]:
#tokenizer = AutoTokenizer.from_pretrained(str(mc.directory), model_max_length=mc.model_max_length)
tokenizer = BertTokenizerFast(
    vocab_file="input/vocab.txt", 
    unk_token="<unk>",
    sep_token="<s>",
    pad_token="<pad>",
    cls_token="<cls>",
    mask_token="<mask>",
    bos_token="<s>",
    eos_token="</s>",
    additional_special_tokens=["click_token", "cart_token", "order_token"],
    model_max_length=mc.model_max_length,
    padding_side="right",
)
tokenizer.add_special_tokens({
    "additional_special_tokens": ["<click>", "<cart>", "<order>"],
})
unk_token = tokenizer.unk_token
unk_id = tokenizer.unk_token_id
pad_token = tokenizer.pad_token
pad_id = tokenizer.pad_token_id
sep_token = tokenizer.sep_token
sep_id = tokenizer.sep_token_id
print(f"{repr(tokenizer)}\nmodel_input_names={tokenizer.model_input_names}")
print(f"{unk_token}={unk_id}\n{pad_token}={pad_id}\n{sep_token}={sep_id}")

PreTrainedTokenizerFast(name_or_path='', vocab_size=1855612, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '<s>', 'pad_token': '<pad>', 'cls_token': '<cls>', 'mask_token': '<mask>', 'additional_special_tokens': ['<click>', '<cart>', '<order>']})
model_input_names=['input_ids', 'token_type_ids', 'attention_mask']
<unk>=3
<pad>=1
<s>=0


In [7]:
%%time
x = tokenizer(
    df["seq"].tolist(),
    truncation=True, 
    padding="max_length",
    stride=mc.stride,
    add_special_tokens=True,
    return_overflowing_tokens=True,
    return_offsets_mapping=False,
    return_special_tokens_mask=False,
)
print(f"{repr(x.keys())}")
overflow_to_sample_mapping = x["overflow_to_sample_mapping"]
print(f"len(overflow_to_sample_mapping)={len(overflow_to_sample_mapping):,}")
#offset_mapping = x["offset_mapping"]
#print(f"len(offset_mapping)={len(offset_mapping):,}")
print(x["input_ids"][0])

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'overflow_to_sample_mapping'])
len(overflow_to_sample_mapping)=129
[5, 6, 23702, 7, 12656, 6, 95742, 6, 382832, 7, 382832, 6, 29147, 6, 382832, 6, 12732, 6, 652875, 6, 78014, 6, 13306, 7, 13306, 6, 78014, 6, 12731, 6, 78014, 6, 78008, 6, 12656, 6, 382832, 6, 611, 6, 12065, 6, 28108, 6, 79910, 6, 12656, 6, 6950, 6, 163586, 6, 47211, 6, 18870, 6, 55840, 7, 55840, 6, 18870, 6, 13919, 6, 19010, 6, 298177, 6, 149149, 6, 811, 6, 94101, 6, 229143, 6, 57743, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [8]:
%%time
input_ids = np.array(x["input_ids"], dtype=np.uint32)
n_pad = (input_ids == tokenizer.pad_token_id).sum()
n_unk = (input_ids == tokenizer.unk_token_id).sum()
n_sep = (input_ids == tokenizer.sep_token_id).sum()
d = input_ids.shape[0] * input_ids.shape[1]
print(f"UNK {n_unk/d*100:.2f}%\t{n_unk:,} out of {d:,} tokens")
print(f"PAD {n_pad/d*100:.2f}%\t{n_pad:,} out of {d:,} tokens")
print(f"SEP {n_sep/d*100:.2f}%\t{n_sep:,} out of {d:,} tokens")
print(f"input_ids.shape={input_ids.shape}")

UNK 0.00%	0 out of 66,048 tokens
PAD 94.14%	62,178 out of 66,048 tokens
SEP 0.20%	129 out of 66,048 tokens
input_ids.shape=(129, 512)
Wall time: 4.31 ms


In [9]:
if conf.sample_frac==1:
    assert x["input_ids"][0] == []
    print("OK")

In [10]:
config = EncoderDecoderConfig.from_encoder_decoder_configs(
    encoder_config=RobertaConfig(
        decoder_start_token_id = tokenizer.cls_token_id,
        pad_token_id = tokenizer.pad_token_id,
        vocab_size = len(tokenizer),
    ),
    decoder_config=RobertaConfig(
        decoder_start_token_id = tokenizer.cls_token_id,
        pad_token_id = tokenizer.pad_token_id,
        vocab_size = len(tokenizer),
    ),
)
#config.decoder_start_token_id = tokenizer.cls_token_id
#config.pad_token_id = tokenizer.pad_token_id
#config.vocab_size = len(tokenizer)
print(config)

EncoderDecoderConfig {
  "decoder": {
    "_name_or_path": "",
    "add_cross_attention": true,
    "architectures": null,
    "attention_probs_dropout_prob": 0.1,
    "bad_words_ids": null,
    "bos_token_id": 0,
    "chunk_size_feed_forward": 0,
    "classifier_dropout": null,
    "cross_attention_hidden_size": null,
    "decoder_start_token_id": 5,
    "diversity_penalty": 0.0,
    "do_sample": false,
    "early_stopping": false,
    "encoder_no_repeat_ngram_size": 0,
    "eos_token_id": 2,
    "exponential_decay_length_penalty": null,
    "finetuning_task": null,
    "forced_bos_token_id": null,
    "forced_eos_token_id": null,
    "hidden_act": "gelu",
    "hidden_dropout_prob": 0.1,
    "hidden_size": 768,
    "id2label": {
      "0": "LABEL_0",
      "1": "LABEL_1"
    },
    "initializer_range": 0.02,
    "intermediate_size": 3072,
    "is_decoder": true,
    "is_encoder_decoder": false,
    "label2id": {
      "LABEL_0": 0,
      "LABEL_1": 1
    },
    "layer_norm_eps": 1e-12

In [11]:
model = EncoderDecoderModel(config=config)
print(model)

EncoderDecoderModel(
  (encoder): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(1855612, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768, padding_idx=1)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Layer

In [12]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")
#print(f"Saved {str(job_dir)}")

Total time taken 0:01:15.342342
